# 7장 문자열 보이어 무어법

In [1]:
256/26

9.846153846153847

In [2]:
256/4

64.0

# 문6) 보이어 무어법이란?
문자열에 패턴을 대고  
패턴의 끝에서부터 왼쪽으로 패턴과 문자열을 비교해 나간다. 이때
    
    A. 문자가 패턴의 끝과 일치하는 경우
        패턴길이가 n일 때 n-k-1만큼 패턴을 오른쪽으로 이동시킨 후, 다시 패턴의 끝과 문자열을 비교 
            (k는 패턴의 끝과 일치한 문자가 패턴에도 나오는 경우,
            패턴내에서 가장 마지막에 등장할 때의 패턴길이상에서의 인덱스값)
                따라서 패턴의 끝과 일치한 문자가 패턴내에 중복되어 있든 아니든, 
                패턴의 끝과 일치한 문자는 패턴내에서 가장 마지막에 등장한 문자이므로
                n-k-1 = 0이 되어 
                패턴을 이동시키지 않고 왼쪽방향으로 일치여부를 비교해나가게 된다.
                계속 비교해서 모두 일치하면 검색성공.
                일치하지 않으면, 아래 "문자가 패턴의 끝과 일치하지 않는 경우B" 에서 판단
                ex) B.1 text DEZCQW
                        pat  ABAC       C에선 일치하는데 A에서 Z와 불일치하고 Z는 패턴에 없으므로
                                ABAC    오른쪽으로 3칸 이동
                ex) B.2 text DWBCQW
                        pat  ABAC        C에서 일치하는데 A에서 B와 불일치하고 하지만 B는 패턴에 있으므로 
                              ABAC       오른쪽으로 1칸 이동  
        
    B. 문자가 패턴의 끝과 일치하지 않는 경우
        1.일치하지 않은 문자가 패턴안에도 없는 경우
         패턴길이가 n일 때 n만큼 패턴을 점프시키고 다시 패턴의 끝과 문자열을 비교
            ex) text WERYERTT
                pat  ABAC       C와 B가 불일치하고 Y는 패턴 내에 없으므로  
                         ABAC   패턴길이 4만큼 이동 
        
        2. 일치하지 않은 문자가 패턴안에 있는 경우
        패턴길이가 n일때 n-k-1 만큼 패턴을 오른쪽으로 이동시킨후, 다시 패턴의 끝과 문자열을 비교
            (k는 패턴의 끝과 일치한 문자가 패턴에도 나오는 경우, 
            패턴내에서 가장 마지막에 등장할 때의 패턴길이상에서의 인덱스값)
            ex) text CABAW
                pat  ABAC
                      ABAC  n-k-1 = 1 따라서 오른쪽으로 1칸 이동
        

# 문7) 보이어무어 구현방법

In [10]:
def bm_match(txt: str, pat: str) -> int:
    """보이어 무어법에 의한 문자열 검색"""
    skip = [None] * 256  # 건너뛰기 표

    # 건너뛰기 표 만들기
    for pt in range(256):
        skip[pt] = len(pat)    # 모든 문자에 대해 패턴의 이동량을 일단 패턴 길이만큼 다 채우고
    for pt in range(len(pat)):
        skip[ord(pat[pt])] = len(pat) - pt - 1  # 패턴의 문자들을 유니코드 숫자로 바꿔서 해당 숫자를 점프테이블의 인덱스로 사용. 
                                                # 패턴이 텍스트에서 패턴에 포함된 문자들을 만났을 때 이동량은
                                                # 패턴 길이-해당 문자 패턴내에서 가장 마지막에 등장할 때의 패턴 길이상에서의 인덱스값-1 
    # 검색하기
    while pt < len(txt):          # 검색 대상 텍스트를 다 볼때까지 반복
        pp = len(pat) - 1         # 패턴의 끝부터 보기 위해 길이-1을 해서 패턴의 마지막 인덱스를 pp가 참조
        while txt[pt] == pat[pp]: # pt도 위 for문의 range 함수 때문에 len(pat)-1 로 초기화 돼 있음 
            if pp == 0:           # 문자와 패턴이 일치할 때, 계속 비교해서 계속 일치하면 검색 성공
                return pt
            pt -= 1
            pp -= 1
        if skip[ord(txt[pt])] > len(pat) - pp:  # 패턴과 문자가 불일치 하면 패턴을 미는데(pt값이 올라가면 패턴이 오른쪽으로 밀리는 것과 같다) 
            pt+=skip[ord(txt[pt])]               # 불일치한 부분의 문자가 패턴에 없으면, skip[ord(txt[pt])] 값은 패턴의 길이와 같고, 그 값은 항상 len(pat)-pp보다 크므로,
                                                  # 비교가 멈춘 곳(패턴과 문자가 불일치한 곳부터)패턴길이만큼 오른쪽으로 밀고
        else:                                   # 불일치한 부분의 문자가 패턴에 있으면, skip[ord(txt[pt])] 값은 len(pat)-pp보다 항상 같거나 작으므로, 
            pt+=len(pat) - pp                    # (패턴길이-해당문자의 패턴상의 인덱스)만큼 다시 말해 (len(pat) - pp)만큼 오른쪽으로 민다. 결과적으로 패턴과 문자가 불일치한 곳, 패턴내 해당 문자가 있는 곳을 일치시키는 것과 같다 
                                                   # 여기서 이동량이 n-k-1 이 아니라 n-k가 되는 이유는?                                                            
    return -1                                                                  

### 코드 작동 흐름
A.B.1 case

    idx  0  1  2  3  4  5
    text D  E  Z  C  W  Q
    pat  A  B  A  C
    
    skip["A"] = 4-2-1 = 1 
    skip["B"] = 4-1-1 = 2
    skip["C"] = 4-3-1 = 0
    skip["나머지알파벳"] = 4
    
    idx  0  1  2  3  4  5  6  7
    text D  E  Z  C  W  Q  T  Y
    pat  A  B  A  C
                  pt
                  pp
               pt
               pp
               skip["Z"] = 4 > len(pat)-pp = 2 따라서 pt=2+4=6
    idx  0  1  2  3  4  5  6  7           
    text D  E  Z  C  W  Q  T  Y
    pat  A  B  A  C                  
                  pp       pt  이건 곧
                  
    text D  E  Z  C  W  Q  T  Y             
    pat           A  B  A  C  오른쪽으로 3칸 민것과 같다.    
    
A.B.2 case    

    idx  0  1  2  3  4  5
    text D  W  B  C  W  Q
    pat  A  B  A  C
    
    skip["A"] = 4-2-1 = 1 
    skip["B"] = 4-1-1 = 2
    skip["C"] = 4-3-1 = 0
    skip["나머지알파벳"] = 4
    
    idx  0  1  2  3  4  5
    text D  W  B  C  W  Q
    pat  A  B  A  C
                  pt
                  pp
               pt
               pp
               skip["B"] = 2 > len(pat)-pp = 2는 틀렸으므로, len(pat)-pp = 2 만큼 pt에 더해준다. 따라서 pt = 4
    idx  0  1  2  3  4  5
    text D  W  B  C  W  Q
    pat  A  B  A  C    
                  pp pt    이건 곧
                  
    idx  0  1  2  3  4  5
    text D  W  B  C  W  Q
    pat     A  B  A  C    오른쪽으로 한칸 민것과 같다.             
    

# 문8) KMP와 보이어무어 비교

패턴의 특성:

    KMP는 패턴 내의 반복되는 부분을 효율적으로 처리할 수 있는 알고리즘입니다. 
    따라서 패턴에 중복되는 부분이 많은 경우 KMP가 더 효율적일 수 있습니다.

    보이어-무어는 패턴의 끝에서 시작하여 문자를 비교합니다.
    패턴이 긴데  패턴에 중복이 적고, 특히 패턴의 끝 부분이 텍스트 내에서 드물게 나타나는 경우 이 알고리즘이 더 빠를 수 있습니다.


텍스트의 특성:

    텍스트가 매우 긴 경우, 보이어-무어 알고리즘이 불일치 발생 시 큰 점프를 통해 검색 시간을 단축시킬 수 있습니다.
    텍스트 내에 패턴과 유사한 부분이 많은 경우, KMP는 부분적인 일치를 더 효율적으로 처리할 수 있습니다.